# Import libraries

In [ ]:
import pandas_datareader as web
import pandas_ta as ta
import pandas as pd
import numpy as np
import yfinance as yf
from yahoo_fin import stock_info as si
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm

# DATA

## Get the investment universe: Dow Jones and Euro Stoxx assets.

In [3]:
dow_tickers = si.tickers_dow() # list of strings of dow tickers
euro_stoxx_pd = pd.read_html('https://en.wikipedia.org/wiki/EURO_STOXX_50')

In [4]:
# the pd[4] contains: Ticker, Main listing, Name, Corporate form, Registered office, Industry, Founded
euro_stoxx = euro_stoxx_pd[4]['Ticker'].tolist()

In [8]:
tickers_list = dow_tickers + euro_stoxx

start_date = pd.to_datetime("2010-08-01")
end_date = start_date + pd.DateOffset(365*13)

df = yf.download(tickers=tickers_list, start=start_date, end=end_date)
df.head()

[*********************100%%**********************]  80 of 80 completed


Price      Adj Close                                                      \
Ticker          AAPL     ABI.BR     AD.AS     ADS.DE ADYEN.AS      AI.PA   
Date                                                                       
2010-08-02  7.888672  30.030293  6.193058  34.836445      NaN  35.758369   
2010-08-03  7.891082  30.139338  6.208486  34.650455      NaN  35.093876   
2010-08-04  7.922717  30.382868  6.186886  35.010025      NaN  35.334049   
2010-08-05  7.884154  29.626844  6.217744  35.675430      NaN  35.310040   
2010-08-06  7.835650  28.485538  6.180714  35.642365      NaN  34.985790   

Price                                                ...      Volume  \
Ticker         AIR.PA     ALV.DE       AMGN    AMZN  ...    STLAM.MI   
Date                                                 ...               
2010-08-02  14.654503  46.920860  38.587025  6.0035  ...  63937674.0   
2010-08-03  14.601805  46.648964  38.474525  6.1210  ...  92567729.0   
2010-08-04  14.707202  46.787483  39.212929  6.3790  ...  65506547.0   
2010-08-05  15.140961  46.253941  38.903500  6.3915  ...  79040453.0   
2010-08-06  15.031508  46.279587  39.339512  6.4160  ...  91511168.0   

Price                                                               \
Ticker          SU.PA        TRV     TTE.PA      UCG.MI        UNH   
Date                                                                 
2010-08-02  2201216.0  4957900.0  6428871.0   7599325.0  7440900.0   
2010-08-03  1842960.0  4927100.0  4572487.0  11724956.0  9954200.0   
2010-08-04  2093440.0  6729900.0  5745183.0  15570602.0  9379500.0   
2010-08-05  2751630.0  1966300.0  6332887.0   8868908.0  7829600.0   
2010-08-06  1981020.0  3140800.0  6942894.0   8489730.0  7479400.0   

Price                                                     
Ticker               V    VOW.DE          VZ         WMT  
Date                                                      
2010-08-02  53846800.0  445434.0  25041100.0  41583000.0  
2010-08-03  34164800.0  364355.0  18411200.0  32005800.0  
2010-08-04  20499200.0  194839.0  17320800.0  24206700.0  
2010-08-05  30787600.0  154969.0  13571000.0  22317000.0  
2010-08-06  19362800.0  211203.0  13053900.0  24810900.0  

[5 rows x 480 columns]

In [9]:
df['Adj Close'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3360 entries, 2010-08-02 to 2023-07-28
Data columns (total 80 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AAPL       3270 non-null   float64
 1   ABI.BR     3326 non-null   float64
 2   AD.AS      3328 non-null   float64
 3   ADS.DE     3301 non-null   float64
 4   ADYEN.AS   1316 non-null   float64
 5   AI.PA      3327 non-null   float64
 6   AIR.PA     3327 non-null   float64
 7   ALV.DE     3301 non-null   float64
 8   AMGN       3270 non-null   float64
 9   AMZN       3270 non-null   float64
 10  ASML.AS    3328 non-null   float64
 11  AXP        3270 non-null   float64
 12  BA         3270 non-null   float64
 13  BAS.DE     3301 non-null   float64
 14  BAYN.DE    3301 non-null   float64
 15  BBVA.MC    3326 non-null   float64
 16  BMW.DE     3301 non-null   float64
 17  BN.PA      3327 non-null   float64
 18  BNP.PA     3327 non-null   float64
 19  CAT        3270 non-null   flo

 4-ADYEN.AS & 59-PRX.AS stocks have the least data

### Data Cleansing

In [10]:
# df.columns = df.columns.str.lower()
df = df.rename(columns={'Adj Close': 'Adj_Close'})
# remove stocks (drops columns) in the DataFrame if all the values in the first row are NaN.
stock_data = df.dropna(axis=1, how='all', subset=df.index[:1]) 
stock_data

Price        Adj_Close                                                \
Ticker            AAPL     ABI.BR      AD.AS      ADS.DE       AI.PA   
Date                                                                   
2010-08-02    7.888672  30.030293   6.193058   34.836445   35.758369   
2010-08-03    7.891082  30.139338   6.208486   34.650455   35.093876   
2010-08-04    7.922717  30.382868   6.186886   35.010025   35.334049   
2010-08-05    7.884154  29.626844   6.217744   35.675430   35.310040   
2010-08-06    7.835650  28.485538   6.180714   35.642365   34.985790   
...                ...        ...        ...         ...         ...   
2023-07-24  191.299622  51.928047  29.777937  174.268692  143.644119   
2023-07-25  192.163055  52.144821  29.678549  180.948318  143.304611   
2023-07-26  193.036453  52.065994  29.773205  176.940536  141.356964   
2023-07-27  191.766098  52.686768  30.042967  181.546509  144.698349   
2023-07-28  194.356461  52.736034  30.109224  184.577255  146.252899   

Price                                                                   ...  \
Ticker          AIR.PA      ALV.DE        AMGN        AMZN     ASML.AS  ...   
Date                                                                    ...   
2010-08-02   14.654503   46.920860   38.587025    6.003500   28.191994  ...   
2010-08-03   14.601805   46.648964   38.474525    6.121000   28.135645  ...   
2010-08-04   14.707202   46.787483   39.212929    6.379000   28.090561  ...   
2010-08-05   15.140961   46.253941   38.903500    6.391500   27.808809  ...   
2010-08-06   15.031508   46.279587   39.339512    6.416000   27.121336  ...   
...                ...         ...         ...         ...         ...  ...   
2023-07-24  135.526505  204.349274  224.910217  128.800003  607.842773  ...   
2023-07-25  132.206924  205.156235  224.805130  129.130005  620.380676  ...   
2023-07-26  132.365936  204.871414  224.709595  128.149994  609.521118  ...   
2023-07-27  130.199265  207.766968  225.512085  128.250000  644.469299  ...   
2023-07-28  132.604477  208.194168  225.817825  132.210007  644.666687  ...   

Price           Volume                                               \
Ticker        STLAM.MI      SU.PA        TRV     TTE.PA      UCG.MI   
Date                                                                  
2010-08-02  63937674.0  2201216.0  4957900.0  6428871.0   7599325.0   
2010-08-03  92567729.0  1842960.0  4927100.0  4572487.0  11724956.0   
2010-08-04  65506547.0  2093440.0  6729900.0  5745183.0  15570602.0   
2010-08-05  79040453.0  2751630.0  1966300.0  6332887.0   8868908.0   
2010-08-06  91511168.0  1981020.0  3140800.0  6942894.0   8489730.0   
...                ...        ...        ...        ...         ...   
2023-07-24   7109803.0   398837.0  1254100.0  3507786.0   9921721.0   
2023-07-25   8897952.0   460629.0  1211800.0  2798807.0  11781748.0   
2023-07-26  22417006.0   935896.0  1782300.0  3848339.0  24299481.0   
2023-07-27  26166721.0  1142855.0  1546400.0  4434934.0  14901682.0   
2023-07-28  20082108.0   937884.0  1352900.0  3948350.0  12690185.0   

Price                                                                
Ticker            UNH           V    VOW.DE          VZ         WMT  
Date                                                                 
2010-08-02  7440900.0  53846800.0  445434.0  25041100.0  41583000.0  
2010-08-03  9954200.0  34164800.0  364355.0  18411200.0  32005800.0  
2010-08-04  9379500.0  20499200.0  194839.0  17320800.0  24206700.0  
2010-08-05  7829600.0  30787600.0  154969.0  13571000.0  22317000.0  
2010-08-06  7479400.0  19362800.0  211203.0  13053900.0  24810900.0  
...               ...         ...       ...         ...         ...  
2023-07-24  2279000.0   6010000.0   22859.0  30784200.0  12529800.0  
2023-07-25  2730600.0   4997500.0   30860.0  35719500.0  15553800.0  
2023-07-26  3697200.0   7551400.0   43123.0  27916000.0  15094200.0  
2023-07-27  3028300.0   8160100.0   76387.0  2809

In [11]:
stock_data['Close'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3360 entries, 2010-08-02 to 2023-07-28
Data columns (total 77 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AAPL       3270 non-null   float64
 1   ABI.BR     3326 non-null   float64
 2   AD.AS      3328 non-null   float64
 3   ADS.DE     3301 non-null   float64
 4   AI.PA      3327 non-null   float64
 5   AIR.PA     3327 non-null   float64
 6   ALV.DE     3301 non-null   float64
 7   AMGN       3270 non-null   float64
 8   AMZN       3270 non-null   float64
 9   ASML.AS    3328 non-null   float64
 10  AXP        3270 non-null   float64
 11  BA         3270 non-null   float64
 12  BAS.DE     3301 non-null   float64
 13  BAYN.DE    3301 non-null   float64
 14  BBVA.MC    3326 non-null   float64
 15  BMW.DE     3301 non-null   float64
 16  BN.PA      3327 non-null   float64
 17  BNP.PA     3327 non-null   float64
 18  CAT        3270 non-null   float64
 19  CRM        3270 non-null   flo

In [12]:
stock_data = stock_data.ffill().stack(-1)# around 90 values are missing so ffill makes the return constant, level = 1 works too here
stock_data
# stock_data.index.names = ['date', 'ticker']

Price               Adj_Close       Close        High         Low        Open  \
Date       Ticker                                                               
2010-08-02 AAPL      7.888672    9.351786    9.378214    9.272143    9.301429   
           ABI.BR   30.030293   41.310001   41.520000   40.544991   40.584999   
           AD.AS     6.193058   11.550707   11.608260   11.430999   11.510421   
           ADS.DE   34.836445   42.145000   42.560001   41.615002   41.810001   
           AI.PA    35.758369   49.000992   49.028419   46.186989   47.624161   
...                       ...         ...         ...         ...         ...   
2023-07-28 UNH     491.560333  502.910004  507.480011  497.279999  505.209991   
           V       233.469498  235.750000  235.880005  233.710007  234.580002   
           VOW.DE  136.733810  146.199997  148.449997  145.050003  146.000000   
           VZ       31.237289   34.029999   34.320000   33.660000   33.669998   
           WMT      52.291199   53.303333   53.646667   53.096668   53.096668   

Price                   Volume  
Date       Ticker               
2010-08-02 AAPL    428055600.0  
           ABI.BR    2022789.0  
           AD.AS     4018944.0  
           ADS.DE     703965.0  
           AI.PA     2351674.0  
...                        ...  
2023-07-28 UNH       2930000.0  
           V         6794800.0  
           VOW.DE      51918.0  
           VZ       30642100.0  
           WMT      11020200.0  

[258720 rows x 6 columns]

In [13]:
print('The number of stocks in the investment universe is:', stock_data['Adj_Close'].loc[stock_data.index[0][0]].count(), 'stocks.')

The number of stocks in the investment universe is: 77 stocks.


## Technical Indicators Calculation

In [14]:
rsi_dayperiod = 20
# level 1 is Ticker index
stock_data[f'RSI_{rsi_dayperiod}'] = stock_data.groupby(level=1)['Adj_Close'].transform(lambda x: ta.rsi(close=x, length=rsi_dayperiod))
# transform() applies a function element-wise on a DataFrame or Series, but it returns an object that is the same shape as the original.
# In the context of a groupby, transform can be used to perform operations that are applied to each group but return the same shape as the original DataFrame
# transform is only allowed to work with a single Series at a time. It is impossible for it to act on two columns at the same time

In [15]:
atr_dayperiod = 15
def atr(stock_data):
    atr = ta.atr(high=stock_data['High'],
                 low=stock_data['Low'],
                 close=stock_data['Adj_Close'],
                 length=atr_dayperiod)
    return atr.sub(atr.mean()).div(atr.std())


# group_keys=False in apply() removes the group names from the resulting index
stock_data[f'ATR_{atr_dayperiod}'] = stock_data.groupby(level=1, group_keys=False).apply(atr)

In [23]:
macd_period = 20
def compute_macd(close):
    macd = ta.macd(close=close, length=macd_period).iloc[:,0] # return MACD days, hours and seconds
    return macd.sub(macd.mean()).div(macd.std())

stock_data[f'MACD_{macd_period}'] = stock_data.groupby(level=1, group_keys=False)['Adj_Close'].apply(compute_macd)

In [24]:
def parkinsonvolatility(stock_data):
    return np.sqrt(1 / (4 * np.log(2)) * (np.log(stock_data['High']) - np.log(stock_data['Low']))**2)

stock_data['Parkinson_Vol'] = stock_data.groupby(level=1, group_keys=False).apply(parkinsonvolatility)

In [25]:
bb_period = 20
# Since both the stock_data and bbands have the same MultiIndex (Date, Ticker), the join aligns the rows correctly.
stock_data =stock_data.join(stock_data.groupby(level=1, group_keys=False)['Adj_Close'].apply(lambda x: ta.bbands(close=x, length=bb_period))
                .iloc[:,0:3].rename(columns={f"BBL_{bb_period}_2.0": f'BBL_{bb_period}', f"BBM_{bb_period}_2.0": f'BBM_{bb_period}',
                                  f"BBU_{bb_period}_2.0": f'BBU_{bb_period}'}))

## Other features

In [26]:
stock_data['Returns'] = stock_data.groupby(level=1)['Adj_Close'].transform(lambda  x: x.pct_change())

In [45]:
print("Around", (stock_data['Volume']==0).sum(), "indexes have 0 volume.")
# inspecting one stock
print(stock_data.xs("NDA-FI.HE",level=1)["Volume"].to_frame().loc["2015-11-02":"2016-01-17"])
zero_Vol = stock_data['Volume'][(stock_data['Volume']<1)].to_frame().index.get_level_values(level=1).unique().tolist()

Around 365 indexes have 0 volume.


In [70]:
# Unstack the 'Volume' column and isolate stocks with zero values (zero_Vol)
volume_data = stock_data.unstack()["Volume"]

# Replace zeros with NaN for only those stocks (zero_Vol) and apply bfill
volume_data[zero_Vol] = volume_data[zero_Vol].replace(0, np.nan).bfill()

# Stack the data back to its original multi-index structure
stock_data["Volume"] = volume_data.stack()
(stock_data['Volume']==0).sum()

In [12]:
norm_period = 10
stock_data[f'Volume_{norm_period}'] = stock_data.groupby(level=1)['Volume'].transform(lambda x: x/x.rolling(window=norm_period).mean())

In [15]:
stock_data

Adj_Close       Close        High         Low        Open  \
Date       Ticker                                                               
2010-08-02 AAPL      7.888672    9.351786    9.378214    9.272143    9.301429   
           ABI.BR   30.030293   41.310001   41.520000   40.544991   40.584999   
           AD.AS     6.193058   11.550707   11.608260   11.430999   11.510421   
           ADS.DE   34.836445   42.145000   42.560001   41.615002   41.810001   
           AI.PA    35.758369   49.000992   49.028419   46.186989   47.624161   
...                       ...         ...         ...         ...         ...   
2023-07-28 UNH     491.560333  502.910004  507.480011  497.279999  505.209991   
           V       233.469498  235.750000  235.880005  233.710007  234.580002   
           VOW.DE  136.733810  146.199997  148.449997  145.050003  146.000000   
           VZ       31.237289   34.029999   34.320000   33.660000   33.669998   
           WMT      52.291199   53.303333   53.646667   53.096668   53.096668   

                        Volume      RSI20     ATR15    MACD20  Parkinson_Vol  \
Date       Ticker                                                              
2010-08-02 AAPL    428055600.0        NaN       NaN       NaN       0.006831   
           ABI.BR    2022789.0        NaN       NaN       NaN       0.014271   
           AD.AS     4018944.0        NaN       NaN       NaN       0.009241   
           ADS.DE     703965.0        NaN       NaN       NaN       0.013485   
           AI.PA     2351674.0        NaN       NaN       NaN       0.035855   
...                        ...        ...       ...       ...            ...   
2023-07-28 UNH       2930000.0  58.601795 -0.108268  2.351794       0.012194   
           V         6794800.0  50.851943 -0.349688  0.497003       0.005550   
           VOW.DE      51918.0  41.937642 -3.233017 -0.451513       0.013915   
           VZ       30642100.0  45.537695 -2.700868 -1.174210       0.011662   
           WMT      11020200.0  63.753251 -2.137489  1.000780       0.006189   

                       BBL_20      BBM_20      BBU_20   Returns  Volume10  
Date       Ticker                                                          
2010-08-02 AAPL           NaN         NaN         NaN       NaN       NaN  
           ABI.BR         NaN         NaN         NaN       NaN       NaN  
           AD.AS          NaN         NaN         NaN       NaN       NaN  
           ADS.DE         NaN         NaN         NaN       NaN       NaN  
           AI.PA          NaN         NaN         NaN       NaN       NaN  
...                       ...         ...         ...       ...       ...  
2023-07-28 UNH     433.702514  473.946048  514.189581 -0.004592  0.753768  
           V       232.212180  237.177266  242.142352  0.005588  1.123592  
           VOW.DE  137.326672  141.922126  146.517580 -0.006118  1.441574  
           VZ       29.193866   31.735115   34.276363  0.014307  0.680519  
           WMT      49.865612   51.268333   52.671054  0.004712  0.813347  

[258720 rows x 15 columns]

In [56]:
# stock_data.to_csv("Daily_EUSTOXX_DOW.csv",index_label=stock_data.index.names)
stock_data = pd.read_csv("Daily_EUSTOXX_DOW.csv", index_col=["Date","Ticker"],parse_dates=['Date'])

### Transform the data to monthly

In [42]:
features_cols = [column for column in stock_data.columns if column not in [ "Returns",'Volume', 'Open', 'High', 'Low', 'Close']]
stock_features = stock_data.unstack()[features_cols].resample('M').last().stack().dropna()
stock_features

Adj_Close      RSI20     ATR15    MACD20  Parkinson_Vol  \
Date       Ticker                                                             
2010-09-30 AAPL      8.548448  65.767954 -1.568820 -0.076333       0.018399   
           ABI.BR   31.367895  55.900891  0.142613  0.252847       0.012038   
           AD.AS     6.102336  45.827753  0.686084 -0.143487       0.008476   
           ADS.DE   37.535244  64.215246 -0.939294  0.162333       0.010943   
           AI.PA    35.822414  54.579483  0.925885  0.267335       0.013393   
...                       ...        ...       ...       ...            ...   
2023-07-31 UNH     491.560333  58.601795 -0.108268  2.351794       0.012194   
           V       233.469498  50.851943 -0.349688  0.497003       0.005550   
           VOW.DE  136.733810  41.937642 -3.233017 -0.451513       0.013915   
           VZ       31.237289  45.537695 -2.700868 -1.174210       0.011662   
           WMT      52.291199  63.753251 -2.137489  1.000780       0.006189   

                       BBL_20      BBM_20      BBU_20  Volume10  
Date       Ticker                                                
2010-09-30 AAPL      7.581182    8.295593    9.010004  1.013680  
           ABI.BR   30.589558   31.229952   31.870345  0.864112  
           AD.AS     6.045295    6.187596    6.329896  2.439811  
           ADS.DE   33.644531   36.086034   38.527537  1.101278  
           AI.PA    34.479280   35.750961   37.022643  0.987160  
...                       ...         ...         ...       ...  
2023-07-31 UNH     433.702514  473.946048  514.189581  0.753768  
           V       232.212180  237.177266  242.142352  1.123592  
           VOW.DE  137.326672  141.922126  146.517580  1.441574  
           VZ       29.193866   31.735115   34.276363  0.680519  
           WMT      49.865612   51.268333   52.671054  0.813347  

[11935 rows x 9 columns]

In [62]:
# Annualize the Returns (Geometric Average)
def calculate_returns(data):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12] # 1-month, 2-month,...

    for lag in lags:

        data[f'Return_{lag}m'] = (data['Adj_Close']
                              .pct_change(lag)
                            #   The pipe() method in pandas allows you to apply a function (or a sequence of functions) to a 
                            #   DataFrame or Series in a clean, readable, and chainable way. It avoids creating temporary variables.
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return data
    
    
stock_features = stock_features.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()
stock_features

Adj_Close      RSI20     ATR15    MACD20  Parkinson_Vol  \
Date       Ticker                                                             
2012-09-30 AAPL     20.183802  51.273943  0.682079 -0.108937       0.012797   
           ABI.BR   50.073990  50.836862  0.892493  0.126900       0.009455   
           AD.AS     6.449656  42.221963  0.315975 -0.398942       0.012017   
           ADS.DE   54.464523  52.950623 -0.348075  0.088686       0.008418   
           AI.PA    44.755360  51.448888  1.113865  0.198290       0.016520   
...                       ...        ...       ...       ...            ...   
2023-07-31 UNH     491.560333  58.601795 -0.108268  2.351794       0.012194   
           V       233.469498  50.851943 -0.349688  0.497003       0.005550   
           VOW.DE  136.733810  41.937642 -3.233017 -0.451513       0.013915   
           VZ       31.237289  45.537695 -2.700868 -1.174210       0.011662   
           WMT      52.291199  63.753251 -2.137489  1.000780       0.006189   

                       BBL_20      BBM_20      BBU_20  Volume10  Return_1m  \
Date       Ticker                                                            
2012-09-30 AAPL     19.750188   20.596856   21.443524  1.098604   0.002796   
           ABI.BR   48.660358   50.635127   52.609897  1.174214  -0.009135   
           AD.AS     6.451245    6.611344    6.771443  0.943169  -0.008444   
           ADS.DE   53.258775   54.786578   56.314381  1.311409   0.026202   
           AI.PA    43.454721   45.096653   46.738586  1.279090   0.030999   
...                       ...         ...         ...       ...        ...   
2023-07-31 UNH     433.702514  473.946048  514.189581  0.753768   0.046334   
           V       232.212180  237.177266  242.142352  1.123592  -0.007285   
           VOW.DE  137.326672  141.922126  146.517580  1.441574  -0.043820   
           VZ       29.193866   31.735115   34.276363  0.680519  -0.068589   
           WMT      49.865612   51.268333   52.671054  0.813347   0.017369   

                   Return_2m  Return_3m  Return_6m  Return_9m  Return_12m  
Date       Ticker                                                          
2012-09-30 AAPL     0.047347   0.046838   0.018680   0.057520    0.048086  
           ABI.BR   0.015464   0.025810   0.035720   0.040498    0.043169  
           AD.AS   -0.007507  -0.000922  -0.004105  -0.002899    0.011701  
           ADS.DE   0.022093   0.041799   0.017324   0.028802    0.029513  
           AI.PA    0.029623   0.023150   0.014781   0.014858    0.018137  
...                      ...        ...        ...        ...         ...  
2023-07-31 UNH      0.018036   0.008653   0.002516  -0.009741   -0.005122  
           V        0.032762   0.004959   0.004625   0.015160    0.009509  
           VOW.DE   0.020446   0.027089   0.004198   0.007728   -0.002992  
           VZ      -0.014002  -0.037354  -0.027284  -0.004863   -0.019724  
           WMT      0.043449   0.020651   0.019111   0.014336    0.017435  

[10087 rows x 15 columns]

## Download Fama-French Factors and Calculate Rolling Factor Betas.

#### Fama-French Five-Factor Model with Momentum: 
$$
R_i - R_f = \alpha_i + \beta_{MKT} (R_m - R_f) + \beta_{SMB} \cdot SMB + \beta_{HML} \cdot HML + \beta_{RMW} \cdot RMW + \beta_{CMA} \cdot CMA + \beta_{MOM} \cdot MOM + \epsilon_i
$$


- $R_i$: Return of asset $i$  
- $R_f$: Risk-free rate  
- $R_m$: Market return  
- $\alpha_i$: Asset-specific intercept (unexplained return component)
- $\epsilon_i$: Error term (idiosyncratic returns)
- $\beta_{MKT}$: Market Risk Premium (MKT or $(R_m - R_f)$)
- $\beta_{SMB}$: Size Factor (SMB - Small Minus Big) [market-cap portfolios]
- $\beta_{HML}$: Value Factor (HML - High Minus Low) [book-to-market ratios]
- $\beta_{RMW}$: Profitability Factor (RMW - Robust Minus Weak) [operating profitability]
- $\beta_{CMA}$: Investment Factor (CMA - Conservative Minus Aggressive) [company’s investment behavior i.e. with low vs high asset growth]
- $\beta_{MOM}$: Momentum factor (UMD - Up Minus Down) [past 11 months returns, excluding the most recent month to avoid short-term reversals]

The model is used to explain stock returns. You can estimate the factor loadings using regression analysis and incorporate them into your portfolio optimization framework.

In [ ]:
ff_mom_df = web.famafrench.FamaFrenchReader('F-F_Momentum_Factor' , freq='M ', start='2011').read()[0]
#! factors are given at the beginning of the month
#! we will resample them as at the end of the month to ease the merge with our data and then shift them later on
ff_mom_df.index = ff_mom_df.index.to_timestamp()
ff_mom_df = ff_mom_df.resample('M').last().div(100)
ff_mom_df.columns = ["MOM"]
ff_mom_df.head()

,MOM
Date,
2011-01-31,-0.0029
2011-02-28,0.0199
2011-03-31,0.0340
2011-04-30,0.0004
2011-05-31,-0.0062


In [106]:
ff_5_factors = web.data.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2011')[0]

ff_5_factors.index = ff_5_factors.index.to_timestamp()

ff_5_factors = ff_5_factors.resample('M').last().div(100)

In [124]:
# .join() is easier for simple index-based joins or when you're dealing with DataFrames with aligned indices.
factor_data = pd.merge(ff_5_factors, ff_mom_df, how="inner" , on="Date").join(stock_features['Return_1m'])

factor_data["Return_1m"] = factor_data["Return_1m"]-factor_data["RF"]
factor_data.drop('RF', axis=1, inplace=True)
factor_data

Mkt-RF     SMB     HML     RMW     CMA     MOM  Return_1m
Date       Ticker                                                           
2012-09-30 AAPL    0.0273  0.0061  0.0160 -0.0149  0.0154 -0.0114   0.002696
           ABI.BR  0.0273  0.0061  0.0160 -0.0149  0.0154 -0.0114  -0.009235
           AD.AS   0.0273  0.0061  0.0160 -0.0149  0.0154 -0.0114  -0.008544
           ADS.DE  0.0273  0.0061  0.0160 -0.0149  0.0154 -0.0114   0.026102
           AI.PA   0.0273  0.0061  0.0160 -0.0149  0.0154 -0.0114   0.030899
...                   ...     ...     ...     ...     ...     ...        ...
2023-07-31 UNH     0.0321  0.0284  0.0411 -0.0057  0.0062 -0.0405   0.041834
           V       0.0321  0.0284  0.0411 -0.0057  0.0062 -0.0405  -0.011785
           VOW.DE  0.0321  0.0284  0.0411 -0.0057  0.0062 -0.0405  -0.048320
           VZ      0.0321  0.0284  0.0411 -0.0057  0.0062 -0.0405  -0.073089
           WMT     0.0321  0.0284  0.0411 -0.0057  0.0062 -0.0405   0.012869

[10087 rows x 7 columns]

#### Get the rolling-factor betas

In [132]:
rolling_window = 12 # 1 year
betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['Return_1m'], 
                                     exog=sm.add_constant(x.drop('Return_1m', axis=1)),
                                     window=rolling_window,
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params)).rename(columns={"const":"ALPHA"})

betas

ALPHA    Mkt-RF       SMB       HML       RMW       CMA  \
Date       Ticker                                                               
2012-09-30 AAPL         NaN       NaN       NaN       NaN       NaN       NaN   
           ABI.BR       NaN       NaN       NaN       NaN       NaN       NaN   
           AD.AS        NaN       NaN       NaN       NaN       NaN       NaN   
           ADS.DE       NaN       NaN       NaN       NaN       NaN       NaN   
           AI.PA        NaN       NaN       NaN       NaN       NaN       NaN   
...                     ...       ...       ...       ...       ...       ...   
2023-07-31 UNH     0.000782  0.822254 -1.481064  1.309523 -0.920016 -0.661280   
           V      -0.009297  1.157252  1.024900 -1.007624  0.415550  1.421210   
           VOW.DE -0.018812  0.312487  2.732243 -1.153819  2.065852 -0.316437   
           VZ     -0.046467 -0.015096  1.925348 -2.056373  1.687449  2.121476   
           WMT     0.015758  0.874208 -1.358183  1.034455 -0.522012 -0.516628   

                        MOM  
Date       Ticker            
2012-09-30 AAPL         NaN  
           ABI.BR       NaN  
           AD.AS        NaN  
           ADS.DE       NaN  
           AI.PA        NaN  
...                     ...  
2023-07-31 UNH     0.368892  
           V      -0.011236  
           VOW.DE  0.329812  
           VZ     -0.461341  
           WMT     0.016989  

[10087 rows x 7 columns]

In [133]:
# factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA','MOM']

#shift the betas by 1 month forward because the return is calculated at the end of the month and the betas are given at the beginning of the month
stock_features = (stock_features.join(betas.groupby(level=1).shift())).drop('Adj_Close', axis=1).dropna()

stock_features.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9163 entries, (Timestamp('2013-09-30 00:00:00'), 'AAPL') to (Timestamp('2023-07-31 00:00:00'), 'WMT')
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   RSI20          9163 non-null   float64
 1   ATR15          9163 non-null   float64
 2   MACD20         9163 non-null   float64
 3   Parkinson_Vol  9163 non-null   float64
 4   BBL_20         9163 non-null   float64
 5   BBM_20         9163 non-null   float64
 6   BBU_20         9163 non-null   float64
 7   Volume10       9163 non-null   float64
 8   Return_1m      9163 non-null   float64
 9   Return_2m      9163 non-null   float64
 10  Return_3m      9163 non-null   float64
 11  Return_6m      9163 non-null   float64
 12  Return_9m      9163 non-null   float64
 13  Return_12m     9163 non-null   float64
 14  ALPHA          9163 non-null   float64
 15  Mkt-RF         9163 non-null   float64
 16  SMB            916

In [2]:
# stock_features.to_csv("Monthly_EUSTOXX_DOW.csv",index_label=stock_data.index.names)
stock_features = pd.read_csv("Monthly_EUSTOXX_DOW.csv", index_col=["Date","Ticker"],parse_dates=['Date'])